## 필요한 모듈 설치 확인

In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from replay_memory import ReplayBuffer, PrioritizedReplayBuffer

import random
import os
import pickle
import time
from copy import deepcopy

# 만약 opencv-python이 설치되어있지 않다면 다음을 통해서 설치해주세요.
# pip install opencv-python
# 만약 설치에 오류가 발생한다면 다음을 참고해주세요.
# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_setup/py_table_of_contents_setup/py_table_of_contents_setup.html#py-table-of-content-setup
from wrappers import wrap

## 하이퍼 파라미터 정의하기

In [2]:
'''DQN settings'''
# sequential images to define state
STATE_LEN = 4
# target policy sync interval
TARGET_REPLACE_ITER = 10**3
# simulator steps for start learning
LEARN_START = 10**4
# (prioritized) experience replay memory size
MEMORY_CAPACITY = 10**6
# simulator steps for learning interval
LEARN_FREQ = 4
# check per
is_per = True
# alpha of PER
PER_ALPHA = 0.6
PER_BETA = 0.4
PER_EPSILON = 1e-6
# Double DQN
DOUBLE = True
# Dueling architecture
DUEL = True

'''Environment Settings'''
# openai gym env name
ENV_NAME = 'PongNoFrameskip-v4'
env = wrap(gym.make(ENV_NAME))
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape
# Total simulation step
STEP_NUM = 10**7
# gamma for MDP
GAMMA = 0.99
# visualize for agent playing
RENDERING = False

'''Training settings'''
# check GPU usage
USE_GPU = torch.cuda.is_available()
print('USE GPU: '+str(USE_GPU))
# mini-batch size
BATCH_SIZE = 32
# learning rage
LR = 1e-4
# epsilon-greedy
EPSILON = 0.0

'''Save&Load Settings'''
# check save/load
SAVE = True
LOAD = False
# save frequency
SAVE_FREQ = 10**5
# paths for predction net, target net, result log
PRED_PATH = './data/model/pred_net.pkl'
TARGET_PATH = './data/model/target_net.pkl'
RESULT_PATH = './data/plots/result.pkl'

USE GPU: True


/home/sungyubkim/gym/gym/logger.py:30: UserWarning: WARN: gym.spaces.Box autodetected dtype as <class 'numpy.uint8'>. Please provide explicit dtype.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/home/sungyubkim/gym/gym/logger.py:30: UserWarning: WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## 네트워크 구조 정의하기

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # nn.Sequential을 사용하면 다음과 같입 코드를 간결하게 바꿀 수 있습니다.
        self.feature_extraction = nn.Sequential(
            nn.Conv2d(STATE_LEN, 32, kernel_size=8, stride=4),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.LeakyReLU(negative_slope=0.01),
        )
        self.fc = nn.Linear(7 * 7 * 64, 256)

        if DUEL:
            # advantage function/ state value function
            self.fc_adv = nn.Linear(256, N_ACTIONS)
            self.fc_val = nn.Linear(256, 1)
        else:
            # action value function
            self.fc_q = nn.Linear(256, N_ACTIONS) 
            
        # 파라미터 값 초기화 코드는 다음과 같이 간결하게 바꿀 수 있습니다.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.orthogonal_(m.weight, gain = np.sqrt(2))
                nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
                nn.init.constant_(m.bias, 0.0)
            

    def forward(self, x):
        # x는 (m, 84, 84, 4)의 tensor
        x = self.feature_extraction(x / 255.0)
        # x.size(0) : mini-batch size
        x = x.view(x.size(0), -1)
        x = F.leaky_relu(self.fc(x), negative_slope=0.01)
        
        if DUEL:
            adv = self.fc_adv(x)
            val = self.fc_val(x)
            action_value = val + adv - adv.mean(1).unsqueeze(1)
        else:
            action_value = self.fc_q(x)

        return action_value

    def save(self, PATH):
        torch.save(self.state_dict(),PATH)

    def load(self, PATH):
        self.load_state_dict(torch.load(PATH))

## DQN 정의하기

In [4]:
class DQN(object):
    def __init__(self):
        self.pred_net, self.target_net = ConvNet(), ConvNet()
        # sync eval target
        self.update_target(self.target_net, self.pred_net, 1.0)
        # use gpu
        if USE_GPU:
            self.pred_net.cuda()
            self.target_net.cuda()
            
        # simulator step conter
        self.memory_counter = 0
        # target network step counter
        self.learn_step_counter = 0
        
        # ceate the replay buffer
        if is_per:
            self.replay_buffer = PrioritizedReplayBuffer(MEMORY_CAPACITY, alpha=PER_ALPHA)
        else:
            self.replay_buffer = ReplayBuffer(MEMORY_CAPACITY)
        
        # define optimizer
        self.optimizer = torch.optim.Adam(self.pred_net.parameters(), lr=LR)
        
    def update_target(self, target, pred, update_rate):
        # update target network parameters using predcition network
        for target_param, pred_param in zip(target.parameters(), pred.parameters()):
            target_param.data.copy_((1.0 - update_rate) \
                                    * target_param.data + update_rate*pred_param.data)
            
    def save_model(self):
        # save prediction network and target network
        self.pred_net.save(PRED_PATH)
        self.target_net.save(TARGET_PATH)

    def load_model(self):
        # load prediction network and target network
        self.pred_net.load(PRED_PATH)
        self.target_net.load(TARGET_PATH)

    def choose_action(self, x, EPSILON):
        x = torch.FloatTensor(x)
        if USE_GPU:
            x = x.cuda()

        if np.random.uniform() < EPSILON:
            # greedy case
            action_value = self.pred_net(x.unsqueeze(0))
            action = torch.argmax(action_value).data.cpu().numpy()
        else:
            # random exploration case
            action = np.random.randint(0, N_ACTIONS)
        return action

    def store_transition(self, s, a, r, s_, done):
        self.memory_counter += 1
        self.replay_buffer.add(s, a, r, s_, float(done))

    def learn(self, beta):
        self.learn_step_counter += 1
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.update_target(self.target_net, self.pred_net, 1.0)
        
        # data sample from experience replay
        if is_per:
            experience = self.replay_buffer.sample(BATCH_SIZE, beta=beta)
            (b_state_memory, b_action_memory, b_reward_memory, \
             b_next_state_memory, b_done, b_weights, b_idxes) = experience
        else:
            b_state_memory, b_action_memory, b_reward_memory, \
            b_next_state_memory, b_done = self.replay_buffer.sample(BATCH_SIZE)
            b_weights, b_idxes = np.ones_like(b_reward_memory), None
            
        b_s = torch.FloatTensor(b_state_memory)
        b_a = torch.LongTensor(b_action_memory)
        b_r = torch.FloatTensor(b_reward_memory)
        b_s_ = torch.FloatTensor(b_next_state_memory)
        b_d = torch.FloatTensor(b_done)

        if USE_GPU:
            b_s, b_a, b_r, b_s_, b_d = b_s.cuda(), b_a.cuda(), b_r.cuda(), b_s_.cuda(), b_d.cuda()

        # action value prediction
        q_eval = self.pred_net(b_s).gather(1, b_a.unsqueeze(1)).view(-1)
        # shape : (m)

        if DOUBLE:
            # get best actions of next state
            _ , best_actions = self.pred_net(b_s_).detach().max(1)
            # get next state value
            q_next = self.target_net(b_s_).detach()
            # get target value
            q_target = b_r + GAMMA *(1.-b_d)* q_next.gather(1, best_actions.unsqueeze(1)).squeeze(1)
            # shape (m)
        else:
            # get next state value
            q_next = self.target_net(b_s_).detach()
            # get target value
            q_target = b_r + GAMMA *(1.-b_d)* q_next.max(1)[0]
            # shape (m)
            
        # calc huber loss, dont reduce for importance weight
        loss = F.smooth_l1_loss(q_eval, q_target, reduction='none')
        # calc importance weighted loss
        loss = torch.mean(torch.Tensor(b_weights).cuda()*loss)
        # get td error
        td_error = (q_target - q_eval).data.cpu().numpy()
        
        # update importance weight
        if is_per:
            new_priorities = np.abs(td_error) + PER_EPSILON
            self.replay_buffer.update_priorities(b_idxes, new_priorities)
        
        # backprop loss
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.pred_net.parameters(),10.)
        self.optimizer.step()

## 학습

In [5]:
dqn = DQN()

# model load with check
if LOAD and os.path.isfile(EVAL_PATH) and os.path.isfile(TARGET_PATH):
    dqn.load_model()
    pkl_file = open(RESULT_PATH,'rb')
    result = pickle.load(pkl_file)
    pkl_file.close()
    print('Load complete!')
else:
    result = []
    print('Initialize results!')

print('Collecting experience...')

# episode step for accumulate reward 
# (since we are using EpisodicLifeEnv of OpenAI gym wrapper)
epi_step = 0
# accumulate return of current episode
entire_ep_r = 0.
# log for accumulate returns
entire_ep_rs = []
# check learning time
start_time = time.time()

while dqn.memory_counter <= STEP_NUM:
    # env reset
    s = np.array(env.reset())
    
    # initialize one episode reward
    ep_r = 0.

    while True:
        a = dqn.choose_action(s, EPSILON)

        # take action and get next state
        s_, r, done, info = env.step(a)
        s_ = np.array(s_)
        
        # accumulate return
        ep_r += r
        # clip rewards for numerical stability
        clip_r = np.sign(r)

        # store the transition
        dqn.store_transition(s, a, clip_r, s_, float(done))

        # annealing the epsilon(exploration strategy), beta(per smoothing)
        if dqn.memory_counter <= MEMORY_CAPACITY:
            # linear annealing to 0.9 until million step
            EPSILON += 0.9/MEMORY_CAPACITY
        elif dqn.memory_counter <= STEP_NUM:
            # linear annealing to 0.99 until the end
            EPSILON += 0.09/(STEP_NUM - MEMORY_CAPACITY)
            # linear annealing to 1 until the end
            PER_BETA += (1.0 - PER_BETA) /(STEP_NUM - MEMORY_CAPACITY)

        # if memory fill 50K and mod 4 = 0(for speed issue), learn pred net
        if (5e+4 <= dqn.memory_counter) and (dqn.memory_counter % LEARN_FREQ == 0):
            dqn.learn(PER_BETA)
            
        # print log and save
        if dqn.memory_counter % SAVE_FREQ == 0:
            # check time interval
            time_interval = round(time.time() - start_time, 2)
            # calc mean return
            mean_10_ep_return = round(np.mean(entire_ep_rs[-10:-1]),2)
            result.append(mean_10_ep_return)
            # print log
            print('Ep: ',epi_step,
                  '| Mean ep 10 return: ', mean_10_ep_return,
                  '/Used Time:',time_interval,
                  '/Used Step:',dqn.memory_counter)
            # save model
            dqn.save_model()
            pkl_file = open(RESULT_PATH, 'wb')
            pickle.dump(np.array(result), pkl_file)
            pkl_file.close()
            print('Save complete!')
            
        # if agent meets end-of-life, update return, acc return
        if done:
            entire_ep_r += ep_r
            epi_step += 1
            if epi_step % 1 == 0:
                entire_ep_rs.append(entire_ep_r)
                entire_ep_r = 0.
            break

        s = s_

        if RENDERING:
            env.render()

Initialize results!


/home/sungyubkim/gym/gym/logger.py:30: UserWarning: WARN: <class 'wrappers.FrameStack'> doesn't implement 'reset' method, but it implements deprecated '_reset' method.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/home/sungyubkim/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Ep:  105 | Mean ep 10 return:  -19.89 /Used Time: 224.92 /Used Step: 100000
Save complete!
Ep:  201 | Mean ep 10 return:  -19.11 /Used Time: 579.91 /Used Step: 200000
Save complete!
Ep:  290 | Mean ep 10 return:  -19.33 /Used Time: 939.82 /Used Step: 300000
Save complete!
Ep:  372 | Mean ep 10 return:  -18.11 /Used Time: 1305.22 /Used Step: 400000
Save complete!
Ep:  441 | Mean ep 10 return:  -17.22 /Used Time: 1675.59 /Used Step: 500000
Save complete!
Ep:  502 | Mean ep 10 return:  -14.89 /Used Time: 2051.08 /Used Step: 600000
Save complete!
Ep:  551 | Mean ep 10 return:  -12.78 /Used Time: 2429.26 /Used Step: 700000
Save complete!
Ep:  593 | Mean ep 10 return:  -8.33 /Used Time: 2813.95 /Used Step: 800000
Save complete!
Ep:  626 | Mean ep 10 return:  -1.0 /Used Time: 3201.12 /Used Step: 900000
Save complete!
Ep:  661 | Mean ep 10 return:  12.56 /Used Time: 3593.12 /Used Step: 1000000
Save complete!
Ep:  699 | Mean ep 10 return:  11.78 /Used Time: 3988.74 /Used Step: 1100000
Save comp

Ep:  4830 | Mean ep 10 return:  20.22 /Used Time: 39929.11 /Used Step: 8900000
Save complete!
Ep:  4889 | Mean ep 10 return:  20.33 /Used Time: 40436.03 /Used Step: 9000000
Save complete!
Ep:  4949 | Mean ep 10 return:  20.0 /Used Time: 40938.02 /Used Step: 9100000
Save complete!
Ep:  5009 | Mean ep 10 return:  19.78 /Used Time: 41425.99 /Used Step: 9200000
Save complete!
Ep:  5069 | Mean ep 10 return:  20.56 /Used Time: 41907.05 /Used Step: 9300000
Save complete!
Ep:  5130 | Mean ep 10 return:  20.22 /Used Time: 42380.33 /Used Step: 9400000
Save complete!
Ep:  5190 | Mean ep 10 return:  20.22 /Used Time: 42854.44 /Used Step: 9500000
Save complete!
Ep:  5249 | Mean ep 10 return:  19.67 /Used Time: 43338.02 /Used Step: 9600000
Save complete!
Ep:  5309 | Mean ep 10 return:  20.11 /Used Time: 43838.64 /Used Step: 9700000
Save complete!
Ep:  5370 | Mean ep 10 return:  20.33 /Used Time: 44352.07 /Used Step: 9800000
Save complete!
Ep:  5429 | Mean ep 10 return:  20.0 /Used Time: 44861.39 /Us